In [3]:
!pip install -U transformers


In [ ]:
!pip install datasets
!pip install soundfile
!pip install librosa

In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [11]:
!pip install speechbrain accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.0/519.0 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.4/519.4 kB 78.1 MB/s eta 0:00:00


## Pre-trained models for text-to-speech

#### SpeechT5

In [ ]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

In [ ]:
inputs = processor(text="Don't count the days, make the days count", return_tensors="pt")

In [ ]:
# load speaker embedding
from datasets import load_dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")


In [ ]:
import torch
speaker_embeddings = torch.tensor(embeddings_dataset[6306]["xvector"]).unsqueeze(0) # change which voice here

In [ ]:
spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)

In [ ]:
# load vocoder to generate audio from log mel spectrogram
from transformers import SpeechT5HifiGan
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

In [ ]:
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [ ]:
import IPython
IPython.display.Audio(speech, rate=16_000)

#### Massive Multilingual Speech (MMS)

In [7]:
#not merged yet so install from branch
!pip install -U git+https://github.com/hollance/transformers.git@vits

  Cloning https://github.com/hollance/transformers.git (to revision vits) to /tmp/pip-req-build-0gh6bqwp
  Running command git clone --filter=blob:none --quiet https://github.com/hollance/transformers.git /tmp/pip-req-build-0gh6bqwp
  Running command git checkout -b vits --track origin/vits
  Switched to a new branch 'vits'
  Branch 'vits' set up to track remote branch 'vits' from 'origin'.
  Resolved https://github.com/hollance/transformers.git to commit 7c21aa2a091bbae0f1b5eca82590287a28c6f555
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [10]:
from transformers import VitsModel

ImportError: cannot import name 'VitsModel' from 'transformers' (/usr/local/lib/python3.9/dist-packages/transformers/__init__.py)

## Fine-tuning SpeechT5


### Housekeeping

In [12]:
from huggingface_hub import notebook_login

notebook_login()

### The dataset

In [13]:
from datasets import load_dataset, Audio

dataset = load_dataset("facebook/voxpopuli", "nl", split="train")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset voxpopuli downloaded and prepared to /root/.cache/huggingface/datasets/facebook___voxpopuli/nl/1.3.0/b5ff837284f0778eefe0f642734e142d8c3f574eba8c9c8a4b13602297f73604. Subsequent calls will reuse this data.


In [14]:
len(dataset)

20968

In [15]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

### Preprocessing the data

In [17]:
from transformers import SpeechT5Processor
checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)

#### Text cleanup for SpeechT5 tokenization

In [18]:
tokenizer = processor.tokenizer

In [19]:
dataset[0]

{'audio_id': '20100210-0900-PLENARY-3-nl_20100210-09:06:43_4',
 'language': 9,
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/02ec6a19d5b97c03e1379250378454dbf3fa2972943504a91c7da5045aa26a89/train_part_0/20100210-0900-PLENARY-3-nl_20100210-09:06:43_4.wav',
  'array': array([ 4.2724609e-04,  1.3122559e-03,  1.0375977e-03, ...,
         -9.1552734e-05,  7.6293945e-04, -2.4414062e-04], dtype=float32),
  'sampling_rate': 16000},
 'raw_text': 'Dat kan naar mijn gevoel alleen met een brede meerderheid die wij samen zoeken.',
 'normalized_text': 'dat kan naar mijn gevoel alleen met een brede meerderheid die wij samen zoeken.',
 'gender': 'female',
 'speaker_id': '1122',
 'is_gold_transcript': True,
 'accent': 'None'}

In [20]:
def extract_all_chars(batch):
    all_text = " ".join(batch["normalized_text"])
    vocab = list(set(all_text))
    return {"vocab":[vocab], "all_text": [all_text]}

In [21]:
vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [22]:
dataset_vocab = set(vocabs["vocab"][0])
tokenizer_vocab = {k for k, _ in tokenizer.get_vocab().items()}

In [23]:
# identify all special characters that tokenizer does not contain
dataset_vocab - tokenizer_vocab

{' ', 'à', 'ç', 'è', 'ë', 'í', 'ï', 'ö', 'ü'}

In [24]:
replacements= [
    ('à', 'a'),
    ('ç', 'c'),
    ('è', 'e'),
    ('ë', 'e'),
    ('í', 'i'),
    ('ï', 'i'),
    ('ö', 'o'),
    ('ü', 'u'),
]

In [25]:
# replace special characters
def cleanup_text(inputs):
    for src, dst in replacements:
        inputs["normalized_text"] = inputs["normalized_text"].replace(src, dst)
    return inputs

dataset = dataset.map(cleanup_text)
    

  0%|          | 0/20968 [00:00<?, ?ex/s]

#### Speakers